<a href="https://colab.research.google.com/github/E-tech-coder/DataScienceCapstoneProject/blob/Elena/RuleBasedSeniority.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
!git clone https://github.com/E-tech-coder/DataScienceCapstoneProject.git

fatal: destination path 'DataScienceCapstoneProject' already exists and is not an empty directory.


In [106]:
import pandas as pd

df = pd.read_csv("/content/DataScienceCapstoneProject/linkedin-cvs-annotatedV5.csv", sep = ";").drop(columns = "linkedin")

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2638 entries, 0 to 2637
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   person_id     2638 non-null   int64 
 1   organization  2638 non-null   object
 2   position      2638 non-null   object
 3   startDate     2520 non-null   object
 4   endDate       1897 non-null   object
 5   status        2638 non-null   object
 6   department    2638 non-null   object
 7   seniority     2638 non-null   object
dtypes: int64(1), object(7)
memory usage: 165.0+ KB


In [108]:
# In the startDate and endDate columns, there are YYYY-MM format, only YYYY and missing values. We need to unify the format.
from datetime import datetime

# if the status is active, then the empty end date is filled with the current year and month.

df.loc[(df["endDate"].isna()) & (df["status"]=="ACTIVE"), "endDate"] = datetime.today().strftime('%Y-%m')


In [109]:
# There are 118 rows without the end date and start date of the job. And their job status is all "UNKNOWN". So it's better to delete these rows.
print(len(df.loc[(df["startDate"].isna() & df["endDate"].isna())]))
set(df.loc[(df["startDate"].isna() & df["endDate"].isna())]["status"].tolist())

118


{'UNKNOWN'}

In [110]:
removed_rows = df.loc[df["startDate"].isna() & df["endDate"].isna() & (df["status"] == "UNKNOWN") ].copy()

In [111]:
remove_idx = df.loc[df["startDate"].isna() & df["endDate"].isna() & (df["status"] == "UNKNOWN") ].index
df = df.drop(remove_idx)

In [112]:
remove_idx

Index([  47,   48,   61,   66,   86,   87,   94,   95,  113,  120,
       ...
       2342, 2343, 2408, 2410, 2412, 2550, 2566, 2567, 2585, 2608],
      dtype='int64', length=118)

In [113]:
type(df.iloc[47]["status"])

str

In [114]:
df.loc[df["startDate"].isna() & df["endDate"].isna()]

,person_id,organization,position,startDate,endDate,status,department,seniority


In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2520 entries, 0 to 2637
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   person_id     2520 non-null   int64 
 1   organization  2520 non-null   object
 2   position      2520 non-null   object
 3   startDate     2520 non-null   object
 4   endDate       2520 non-null   object
 5   status        2520 non-null   object
 6   department    2520 non-null   object
 7   seniority     2520 non-null   object
dtypes: int64(1), object(7)
memory usage: 177.2+ KB


In [116]:
set(df["endDate"].str.len().tolist())

{4, 7}

In [117]:
#Since there's no null values anymore, we will turn the start date and end date into datetime format.

df["endDate"] = pd.to_datetime( df["endDate"], format="mixed")


In [118]:
# Keep the original value where condition is True, otherwise replace it.
df["startDate"] = pd.to_datetime(
    df["startDate"].where(df["startDate"].str.len() == 7, df["startDate"]+"-01"),
    format = "mixed"
)

In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2520 entries, 0 to 2637
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   person_id     2520 non-null   int64         
 1   organization  2520 non-null   object        
 2   position      2520 non-null   object        
 3   startDate     2520 non-null   datetime64[ns]
 4   endDate       2520 non-null   datetime64[ns]
 5   status        2520 non-null   object        
 6   department    2520 non-null   object        
 7   seniority     2520 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 177.2+ KB


In [120]:
df["WorkYears"] = round((df["endDate"]-df["startDate"]).dt.days /365, 3)

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2520 entries, 0 to 2637
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   person_id     2520 non-null   int64         
 1   organization  2520 non-null   object        
 2   position      2520 non-null   object        
 3   startDate     2520 non-null   datetime64[ns]
 4   endDate       2520 non-null   datetime64[ns]
 5   status        2520 non-null   object        
 6   department    2520 non-null   object        
 7   seniority     2520 non-null   object        
 8   WorkYears     2520 non-null   float64       
dtypes: datetime64[ns](2), float64(1), int64(1), object(5)
memory usage: 196.9+ KB


In [124]:
set(df["seniority"].tolist())

{'Director', 'Junior', 'Lead', 'Management', 'Professional', 'Senior'}

In [129]:
df[df["person_id"]==0]

,person_id,organization,position,startDate,endDate,status,department,seniority,WorkYears
0,0,Depot4Design GmbH,Prokurist,2019-08-01,2025-12-01,ACTIVE,Other,Management,6.340
1,0,Depot4Design GmbH,CFO,2019-07-01,2025-12-01,ACTIVE,Other,Management,6.425
2,0,Depot4Design GmbH,Betriebswirtin,2019-07-01,2025-12-01,ACTIVE,Other,Professional,6.425
3,0,Depot4Design GmbH,Prokuristin,2019-07-01,2025-12-01,ACTIVE,Other,Management,6.425
4,0,Depot4Design GmbH,CFO,2019-07-01,2025-12-01,ACTIVE,Other,Management,6.425
5,0,Nagel Car Group,Buchhalterin,2000-05-01,2019-06-01,INACTIVE,Other,Professional,19.096


In [122]:
df.head(200)

,person_id,organization,position,startDate,endDate,status,department,seniority,WorkYears
0,0,Depot4Design GmbH,Prokurist,2019-08-01,2025-12-01,ACTIVE,Other,Management,6.340
1,0,Depot4Design GmbH,CFO,2019-07-01,2025-12-01,ACTIVE,Other,Management,6.425
2,0,Depot4Design GmbH,Betriebswirtin,2019-07-01,2025-12-01,ACTIVE,Other,Professional,6.425
3,0,Depot4Design GmbH,Prokuristin,2019-07-01,2025-12-01,ACTIVE,Other,Management,6.425
4,0,Depot4Design GmbH,CFO,2019-07-01,2025-12-01,ACTIVE,Other,Management,6.425
...,...,...,...,...,...,...,...,...,...
208,57,ATECNA Montréal,"Associate Director, Data Strategy & Optimisation",2024-01-01,2025-01-01,INACTIVE,Information Technology,Director,1.003
209,57,ATECNA Montréal,General Manager,2021-01-01,2024-01-01,INACTIVE,Other,Management,3.000
210,57,ATECNA Montréal,"Associate Director, Digital Analytics",2020-09-01,2021-10-01,INACTIVE,Marketing,Director,1.082
211,57,ATECNA Montréal,Digital Performance Director,2019-11-01,2020-08-01,INACTIVE,Marketing,Director,0.751


In [123]:
df.head()

,person_id,organization,position,startDate,endDate,status,department,seniority,WorkYears
0,0,Depot4Design GmbH,Prokurist,2019-08-01,2025-12-01,ACTIVE,Other,Management,6.340
1,0,Depot4Design GmbH,CFO,2019-07-01,2025-12-01,ACTIVE,Other,Management,6.425
2,0,Depot4Design GmbH,Betriebswirtin,2019-07-01,2025-12-01,ACTIVE,Other,Professional,6.425
3,0,Depot4Design GmbH,Prokuristin,2019-07-01,2025-12-01,ACTIVE,Other,Management,6.425
4,0,Depot4Design GmbH,CFO,2019-07-01,2025-12-01,ACTIVE,Other,Management,6.425


In [130]:
seniority_df = pd.read_csv('/content/DataScienceCapstoneProject/seniority-v2.csv')

In [134]:
import re
BOW_seniority = {}

for label in set(seniority_df["label"].tolist()):
  x = []
  for text in seniority_df[seniority_df["label"]==label]["text"]:
    tokens = re.split(r"[ ./&()|-]", text.lower().strip())
    tokens = [t for t in tokens if t]
    x.extend(tokens)
  BOW_seniority[label] = list(x)

In [ ]:
BOW_seniority

In [143]:
titles = []
corpus = []
for title, cor in BOW_seniority.items():
  titles.append(title)
  corpus.append(cor)


In [145]:
titles

['Lead', 'Senior', 'Director', 'Management', 'Junior']

In [147]:
from numpy import vectorize

# Cosine similarity

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# turn seperated words into a string sentence in each corpus

corpus = [" ".join(doc) for doc in corpus]

vectorizer = TfidfVectorizer(ngram_range=(1,3))
vector = vectorizer.fit_transform(corpus)

# Since we have fit the vectorize before, we only need to transform it.

# X_positions = vectorizer.transform(df['position'])
# similarity = cosine_similarity(X_positions, tfidf_df)

# import numpy as np
# df["cosine_predict"] = tfidf_df.index[similarity.argmax(axis = 1)]
# df["cosine_ifidf_score"] = similarity.max(axis = 1)


In [153]:
import numpy as np

X_positions = vectorizer.transform(df["position"])
similarity = cosine_similarity(X_positions, tfidf_df)

In [157]:
df["TFIDF-seniority"]=tfidf_df.index[similarity.argmax(axis=1)]

In [156]:
similarity.shape

(2520, 5)

In [152]:
tfidf_df = pd.DataFrame(vector.toarray(), index = titles, columns = vectorizer.get_feature_names_out())

In [160]:
# Calculate accuracy using only text analysis without considering work years
len(df[df["seniority"]==df["TFIDF-seniority"]])/len(df)

0.34404761904761905